<a href="https://colab.research.google.com/github/minnji88/NLP-study/blob/main/sentencepiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.3MB/s 


In [14]:
import pandas as pd
import sentencepiece as spm
import urllib.request

In [15]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7fa8813f7470>)

In [16]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [17]:
print('리뷰 개수 :',len(naver_df))

리뷰 개수 : 200000


DataFrame에 NaN값이 있는지 알고 싶다면 `values.any()` 메서드를 사용할 수 있습니다. 

DataFrame에 NaN항목이 하나도 없으면 False입니다.

In [18]:
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

True


In [19]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [20]:
print('리뷰 개수 :',len(naver_df))

리뷰 개수 : 199992


In [21]:
input_file = 'naver_review.txt'

with open(input_file, 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [23]:
templates = '--input={} --model_prefix={} --vocab_size={} --max_sentence_length={}'

prefix = 'naver'
vocab_size = 5000
max_len = 512 # 문자 단위 (김민지  = 3)

cmd = templates.format(input_file, prefix, vocab_size, max_len)

# 학습이 잘 완료되면 True라는 인자를 반환합니다.
# 이부분이 완료되면, prefix를 가진 아래의 두개의 파일이 생성됩니다
spm.SentencePieceTrainer.Train(cmd)

In [26]:
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(prefix))

True

In [27]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line)) # EncodeAsPieces : string으로 tokenize
  print(sp.encode_as_ids(line)) # EncodeAsIds : text => id(해당 인덱스 반환)
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화', '냐', '.']
[165, 968, 1639, 8, 119, 4]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[39, 148, 840, 152]



In [28]:
with open('{}.vocab'.format(prefix), encoding='utf-8') as f:
    vocabs = [doc.strip() for doc in f]

print('num of vocabs = {}'.format(len(vocabs)))

num of vocabs = 5000


In [29]:
vocabs[:10]

['<unk>\t0',
 '<s>\t0',
 '</s>\t0',
 '▁\t-3.01183',
 '.\t-3.57832',
 '이\t-4.21137',
 '..\t-4.48333',
 '가\t-4.58898',
 '▁영화\t-4.62769',
 '의\t-4.63297']

In [35]:
# returns vocab size
sp.GetPieceSize()

5000

In [36]:
# id <=> piece conversion
sp.IdToPiece(8)

'▁영화'

In [32]:
# returns id for '_영화'
sp.PieceToId('▁영화')

8

In [33]:
# decoded to <sep>
sp.DecodeIds([39, 148, 840, 152]) 

'진짜 최고의 영화입니다 ᄏᄏ'

In [34]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[39, 148, 840, 152]
